**Общая информация**

**Срок сдачи:** 29 марта 2021, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml, fetch_20newsgroups # can't import fetch_mldata

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from scipy.spatial import distance

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [62]:
class MyKNeighborsClassifier:
    def __init__(self, n_neighbors=2):
        self.n_neighbors = n_neighbors
        self.X = []
        self.y = []
        
    def fit(self, X, y):
        self.X = np.array(X)
        self.y = np.array(y)
        
    def _class(self, new_point): 
        diff_squared = (self.X - new_point) ** 2
        euclidean_distances = np.sqrt(diff_squared.sum(axis=1)) # minkowski, p=2
        sorted_dist_indices = euclidean_distances.argsort()[:self.n_neighbors]
                
        class_count = {}
        for i in sorted_dist_indices:
            class_label = self.y[i]
            class_count[class_label] = class_count.get(class_label, 0) + 1
            
        sorted_class_count = sorted(class_count.items(),
                                   key=lambda kv: (kv[1], kv[0]),
                                   reverse=True)
        
        return sorted_class_count[0][0]
    
    
    def predict(self, X_test):
        my_clf_pred = []
        for point in X_test:
            my_clf_pred.append(self._class(point))
        
        return np.array(my_clf_pred)

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [63]:
iris = datasets.load_iris()

In [64]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [65]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
# my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [66]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [67]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [70]:
%time clf.fit(X_train, y_train)

CPU times: user 1.06 ms, sys: 312 µs, total: 1.37 ms
Wall time: 1.38 ms


KNeighborsClassifier(algorithm='brute', n_neighbors=2)

In [72]:
%time my_clf.fit(X_train, y_train)

CPU times: user 42 µs, sys: 33 µs, total: 75 µs
Wall time: 60.1 µs


In [73]:
%time clf.predict(X_test)

CPU times: user 3.79 ms, sys: 1.01 ms, total: 4.8 ms
Wall time: 3.95 ms


array([0, 1, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1])

In [74]:
%time my_clf.predict(X_test)

CPU times: user 2.27 ms, sys: 2.5 ms, total: 4.77 ms
Wall time: 3.7 ms


array([0, 1, 0, 1, 2, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [76]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
# my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [78]:
%time clf.fit(X_train, y_train)

CPU times: user 2.62 ms, sys: 5.81 ms, total: 8.43 ms
Wall time: 29.6 ms


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [79]:
%time my_clf.fit(X_train, y_train)

CPU times: user 36 µs, sys: 34 µs, total: 70 µs
Wall time: 59.8 µs


In [80]:
%time clf.predict(X_test)

CPU times: user 6.77 ms, sys: 20.4 ms, total: 27.2 ms
Wall time: 60.5 ms


array([1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 2, 2, 2, 2])

In [81]:
%time my_clf.predict(X_test)

CPU times: user 1.78 ms, sys: 138 µs, total: 1.92 ms
Wall time: 2.51 ms


array([1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 2, 2, 2, 2])

In [82]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [ ]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [ ]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.